In [13]:
import os

os.environ["EMAIL_USER"] = "badabagni25r@gmail.com"
os.environ["EMAIL_PASS"] = "yjcm lhxj kfap lsvq"


In [17]:
import os, json

config = {
    "enabled_channels": {
        "console": True,
        "email": True,  # enable email alerts now if you want
        "gui": False
    },
    "email": {
        "smtp_server": "smtp.gmail.com",
        "smtp_port": 587,
        "sender": os.getenv("EMAIL_USER"),
        "recipients": ["rohi131279@gmail.com"],
        "username": os.getenv("EMAIL_USER"),
        "password": os.getenv("EMAIL_PASS")
    },
    "suppression_window_seconds": 30,
    "outputs_dir": "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs",
    "log_csv": "alert_log.csv",
    "log_text": "alert_summary.txt",
    "alert_timeout_seconds": 2,
    "max_alerts_per_minute": 60
}

In [3]:
import smtplib
from email.mime.text import MIMEText
import os

sender = os.getenv("EMAIL_USER")
password = os.getenv("EMAIL_PASS")
recipient = sender  # send to yourself for testing

msg = MIMEText("Test alert: Your PPE detection system email alerts are working!")
msg["Subject"] = "PPE Detection Alert Test"
msg["From"] = sender
msg["To"] = recipient

try:
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender, password)
        server.send_message(msg)
    print("Test email sent successfully!")
except Exception as e:
    print("Failed to send email:", e)


Test email sent successfully!


In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s eta 0:00:00


In [6]:
!find /content -name "alert_manager.py"

In [7]:
import sys
sys.path.append("/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system")


In [8]:
from alert_manager import AlertManager


In [9]:
import os, csv, json, cv2, time, psutil, collections
from datetime import datetime, timezone
from ultralytics import YOLO
from alert_manager import AlertManager

# ---------------- CONFIG ----------------
MODEL_PATH = "/content/drive/MyDrive/Project/Dataset/css-data/cross_validation/results_fold3/weights/best.pt"
VIDEO_PATH = "/content/drive/MyDrive/Project/Dataset/finalTest.mp4"
OUTPUT_DIR = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/visual_demo/outputs/alerts"
ALERT_BASE = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs/metaData"

VIOLATION_FRAMES_DIR = os.path.join(ALERT_BASE, "violations", "frames")
VIOLATION_CLIPS_DIR = os.path.join(ALERT_BASE, "violations", "clips")
METADATA_CSV = os.path.join(ALERT_BASE, "detection_metadata.csv")
METADATA_JSON = os.path.join(ALERT_BASE, "detection_metadata.json")
PERF_LOG = os.path.join(ALERT_BASE, "performance_log.csv")

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(VIOLATION_FRAMES_DIR, exist_ok=True)
os.makedirs(VIOLATION_CLIPS_DIR, exist_ok=True)
os.makedirs(os.path.dirname(METADATA_CSV), exist_ok=True)

FRAME_SKIP = 3
RESIZE_WIDTH = 1280
CLIP_FRAMES = 30
# ---------------------------------------

# Initialize model and alert manager
model = YOLO(MODEL_PATH)
alert_mgr = AlertManager()
base_name = os.path.basename(VIDEO_PATH)

cap = cv2.VideoCapture(VIDEO_PATH)
fps = int(cap.get(cv2.CAP_PROP_FPS)) or 15
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
new_h = int(h * (RESIZE_WIDTH / w))

out_path = os.path.join(OUTPUT_DIR, f"alert_{base_name}")
out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (RESIZE_WIDTH, new_h))

# Metadata & performance files
if not os.path.exists(METADATA_CSV):
    with open(METADATA_CSV, "w", newline="") as f:
        csv.writer(f).writerow(["timestamp", "video", "frame", "worker_id", "violation", "confidence"])

if not os.path.exists(PERF_LOG):
    with open(PERF_LOG, "w", newline="") as f:
        csv.writer(f).writerow(["timestamp", "frame_id", "latency_sec", "fps", "cpu_percent", "mem_mb"])

metadata_rows = []
metadata_json = []
perf_data = []

buffer = collections.deque(maxlen=CLIP_FRAMES)
frame_id, processed = 0, 0
start_time = time.time()
print(f"Processing video: {base_name}")

# ---------------- MAIN LOOP ----------------
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_id += 1
    if frame_id % FRAME_SKIP != 0:
        continue

    frame = cv2.resize(frame, (RESIZE_WIDTH, new_h))
    timestamp = datetime.now(timezone.utc).isoformat()
    buffer.append((frame_id, frame.copy()))

    t0 = time.time()
    results = model(frame, verbose=False)
    latency = time.time() - t0

    annotated = results[0].plot()
    names = model.names
    detections = [(names[int(box.cls[0])], float(box.conf[0])) for box in results[0].boxes]
    violations = [(lbl, conf) for lbl, conf in detections if "NO-" in lbl]

    # Log violations
    if violations:
        for v_lbl, v_conf in violations:
            worker_id = f"{base_name}_F{frame_id}_{v_lbl}"
            metadata_rows.append([timestamp, base_name, frame_id, worker_id, v_lbl, round(v_conf, 3)])
            metadata_json.append({
                "timestamp": timestamp,
                "video": base_name,
                "frame": frame_id,
                "worker_id": worker_id,
                "violation": v_lbl,
                "confidence": round(v_conf, 3)
            })

        event = {
            "video": base_name,
            "frame": frame_id,
            "violations": [v[0] for v in violations],
            "timestamp": timestamp
        }
        alert_mgr.handle_violation(event)

        # Save violation frame & clip
        cv2.imwrite(os.path.join(VIOLATION_FRAMES_DIR, f"{base_name}_F{frame_id}.jpg"), annotated)
        clip_path = os.path.join(VIOLATION_CLIPS_DIR, f"{base_name}_F{frame_id}_clip.mp4")
        clip_writer = cv2.VideoWriter(clip_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (RESIZE_WIDTH, new_h))
        for _, fr in list(buffer):
            clip_writer.write(fr)
        clip_writer.release()

    out.write(annotated)
    processed += 1

    # ---------- Performance logging ----------
    cpu = psutil.cpu_percent(interval=None)
    mem = psutil.virtual_memory().used / (1024 ** 2)
    elapsed = time.time() - start_time
    fps_calc = processed / elapsed
    perf_data.append([timestamp, frame_id, round(latency, 3), round(fps_calc, 2), cpu, round(mem, 2)])

    if processed % 50 == 0:
        print(f"Processed {processed} frames | FPS: {fps_calc:.2f} | CPU: {cpu:.1f}% | MEM: {mem:.1f}MB | Latency: {latency:.3f}s")

# ---------------- SAVE LOGS ----------------
cap.release()
out.release()
cv2.destroyAllWindows()

with open(METADATA_CSV, "a", newline="") as f:
    csv.writer(f).writerows(metadata_rows)

with open(METADATA_JSON, "w") as f:
    json.dump(metadata_json, f, indent=2)

with open(PERF_LOG, "a", newline="") as f:
    csv.writer(f).writerows(perf_data)

# ---------------- SUMMARY ----------------
latencies = [p[2] for p in perf_data]
avg_latency = sum(latencies)/len(latencies)
avg_fps = sum([p[3] for p in perf_data])/len(perf_data)
avg_cpu = sum([p[4] for p in perf_data])/len(perf_data)
avg_mem = sum([p[5] for p in perf_data])/len(perf_data)

print("\nProcessing complete!")
print(f"Output video: {out_path}")
print(f"Metadata CSV: {METADATA_CSV}")
print(f"Metadata JSON: {METADATA_JSON}")
print(f"Performance log: {PERF_LOG}")
print(f"\n--- PERFORMANCE SUMMARY ---")
print(f"Average FPS: {avg_fps:.2f}")
print(f"Average Latency: {avg_latency:.3f}s/frame")
print(f"Average CPU Usage: {avg_cpu:.1f}%")
print(f"Average Memory Usage: {avg_mem:.1f}MB")
print(f"Alert logs in: {alert_mgr.cfg['outputs_dir']}")


Processing video: finalTest.mp4


[2025-11-08 10:19:20,983] WARNING: ALERT - finalTest.mp4 Frame:3 Violations:['NO-Mask', 'NO-Mask', 'NO-Safety Vest'] Timestamp:2025-11-08T10:19:20.983810+00:00
[2025-11-08 10:19:35,162] WARNING: ALERT - finalTest.mp4 Frame:93 Violations:['NO-Mask', 'NO-Mask', 'NO-Safety Vest', 'NO-Safety Vest'] Timestamp:2025-11-08T10:19:35.162471+00:00
[2025-11-08 10:19:39,713] WARNING: ALERT - finalTest.mp4 Frame:117 Violations:['NO-Mask', 'NO-Mask', 'NO-Safety Vest', 'NO-Safety Vest', 'NO-Safety Vest'] Timestamp:2025-11-08T10:19:39.713597+00:00
[2025-11-08 10:19:44,652] WARNING: ALERT - finalTest.mp4 Frame:147 Violations:['NO-Mask', 'NO-Safety Vest', 'NO-Safety Vest', 'NO-Safety Vest'] Timestamp:2025-11-08T10:19:44.652784+00:00
[2025-11-08 10:19:45,164] WARNING: ALERT - finalTest.mp4 Frame:150 Violations:['NO-Safety Vest', 'NO-Mask'] Timestamp:2025-11-08T10:19:45.164732+00:00


Processed 50 frames | FPS: 1.99 | CPU: 62.2% | MEM: 1655.4MB | Latency: 0.263s


[2025-11-08 10:19:45,672] WARNING: ALERT - finalTest.mp4 Frame:153 Violations:['NO-Mask', 'NO-Safety Vest', 'NO-Safety Vest'] Timestamp:2025-11-08T10:19:45.672180+00:00
[2025-11-08 10:19:53,695] WARNING: ALERT - finalTest.mp4 Frame:198 Violations:['NO-Mask', 'NO-Safety Vest', 'NO-Mask'] Timestamp:2025-11-08T10:19:53.695205+00:00
[2025-11-08 10:20:05,628] WARNING: ALERT - finalTest.mp4 Frame:267 Violations:['NO-Safety Vest', 'NO-Mask', 'NO-Hardhat'] Timestamp:2025-11-08T10:20:05.628108+00:00
[2025-11-08 10:20:07,145] WARNING: ALERT - finalTest.mp4 Frame:276 Violations:['NO-Safety Vest', 'NO-Mask', 'NO-Safety Vest', 'NO-Mask'] Timestamp:2025-11-08T10:20:07.145614+00:00
[2025-11-08 10:20:09,617] WARNING: ALERT - finalTest.mp4 Frame:291 Violations:['NO-Safety Vest', 'NO-Mask', 'NO-Mask', 'NO-Mask'] Timestamp:2025-11-08T10:20:09.617248+00:00


Processed 100 frames | FPS: 1.96 | CPU: 60.8% | MEM: 1652.7MB | Latency: 0.256s


[2025-11-08 10:20:13,102] WARNING: ALERT - finalTest.mp4 Frame:309 Violations:['NO-Safety Vest', 'NO-Mask', 'NO-Mask', 'NO-Safety Vest', 'NO-Safety Vest'] Timestamp:2025-11-08T10:20:13.102736+00:00



Processing complete!
Output video: /content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/visual_demo/outputs/alerts/alert_finalTest.mp4
Metadata CSV: /content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs/metaData/detection_metadata.csv
Metadata JSON: /content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs/metaData/detection_metadata.json
Performance log: /content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs/metaData/performance_log.csv

--- PERFORMANCE SUMMARY ---
Average FPS: 1.93
Average Latency: 0.275s/frame
Average CPU Usage: 65.6%
Average Memory Usage: 1642.9MB
Alert logs in: /content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs


In [10]:
import os
print("EMAIL_USER =", os.getenv("EMAIL_USER"))
print("EMAIL_PASS =", os.getenv("EMAIL_PASS"))


EMAIL_USER = badabagni25r@gmail.com
EMAIL_PASS = yjcm lhxj kfap lsvq


In [18]:
from email_alert import send_email_alert

test_event = {
    "video": "test_video.mp4",
    "frame": 42,
    "violations": ["NO-HELMET", "NO-VEST"],
    "timestamp": datetime.now(timezone.utc).isoformat()
}

import json, os
from datetime import datetime, timezone

email_cfg = {
    "smtp_server": "smtp.gmail.com",
    "smtp_port": 587,
    "sender": os.getenv("EMAIL_USER"),
    "recipients": ["rohi131279@gmail.com"],
    "username": os.getenv("EMAIL_USER"),
    "password": os.getenv("EMAIL_PASS")
}

result = send_email_alert(test_event, email_cfg)
print(result)


{'status': 'sent', 'channel': 'email', 'ts': '2025-11-08T10:43:33.520914+00:00'}


In [21]:
import os, json

config = {
    "enabled_channels": {
        "console": True,
        "email": True,  # enable email alerts now if you want
        "gui": False
    },
    "email": {
        "smtp_server": "smtp.gmail.com",
        "smtp_port": 587,
        "sender": os.getenv("EMAIL_USER"),
        "recipients": ["rohi131279@gmail.com"],
        "username": os.getenv("EMAIL_USER"),
        "password": os.getenv("EMAIL_PASS")
    },
    "suppression_window_seconds": 30,
    "outputs_dir": "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs",
    "log_csv": "alert_log.csv",
    "log_text": "alert_summary.txt",
    "alert_timeout_seconds": 2,
    "max_alerts_per_minute": 60
}
import smtplib
from email.message import EmailMessage
from datetime import datetime, timezone

def send_email_alert(violation_event, email_cfg):
    # email_cfg must include smtp_server, smtp_port, sender, recipients(list), username, password
    msg = EmailMessage()
    subject = f"PPE Violation: {', '.join(violation_event['violations'])} - {violation_event['video']}"
    body = f"""PPE Violation detected.
Video: {violation_event['video']}
Frame: {violation_event['frame']}
Violations: {', '.join(violation_event['violations'])}
Timestamp (UTC): {violation_event['timestamp']}
-- Alert System"""
    msg["From"] = email_cfg.get("sender")
    msg["To"] = ", ".join(email_cfg.get("recipients", []))
    msg["Subject"] = subject
    msg.set_content(body)

    try:
        with smtplib.SMTP(email_cfg["smtp_server"], email_cfg["smtp_port"], timeout=10) as server:
            server.ehlo()
            server.starttls()
            server.login(email_cfg["username"], email_cfg["password"])
            server.send_message(msg)
        return {"status": "sent", "channel": "email", "ts": datetime.now(timezone.utc).isoformat()}
    except Exception as e:
        return {"status": "failed", "channel": "email", "error": str(e), "ts": datetime.now(timezone.utc).isoformat()}
import logging
from datetime import datetime, timezone

logger = logging.getLogger("alert_system")
logger.setLevel(logging.INFO)
if not logger.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(asctime)s] %(levelname)s: %(message)s")
    ch.setFormatter(formatter)
    logger.addHandler(ch)

def send_console_alert(violation_event):
    ts = datetime.now(timezone.utc).isoformat()
    msg = f"ALERT - {violation_event['video']} Frame:{violation_event['frame']} Violations:{violation_event['violations']} Timestamp:{ts}"
    logger.warning(msg)
    return {"status": "sent", "channel": "console", "ts": ts}

In [22]:
import os
from ultralytics import YOLO
import cv2
from datetime import datetime, timezone
from alert_manager import AlertManager

# Initialize model and alert system
model = YOLO("/content/drive/MyDrive/Project/Dataset/css-data/cross_validation/results_fold3/weights/best.pt")
alert_mgr = AlertManager()

# Input video path
video_path = "/content/drive/MyDrive/Project/Dataset/finalTest.mp4"
base_name = os.path.basename(video_path)

# Output directory
output_dir = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/visual_demo/videos"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"alert_{base_name}")

# Read video
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Video writer
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

frame_id = 0
print(f"Processing video: {base_name}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame, verbose=False)

    annotated_frame = results[0].plot()
    names = model.names
    detections = [names[int(cls)] for cls in results[0].boxes.cls]

    violations = [v for v in detections if "NO-" in v]  # rule-based

    # Send alert if violations exist
    if violations:
        event = {
            "video": base_name,
            "frame": frame_id,
            "violations": violations,
            "timestamp": datetime.now(timezone.utc).isoformat()
        }
        alert_mgr.handle_violation(event)

    out.write(annotated_frame)

cap.release()
out.release()

print(f"\n✅ Video processed and saved at: {output_path}")
print(f"📄 Alert logs stored in: {alert_mgr.cfg['outputs_dir']}")


Processing video: finalTest.mp4


[2025-11-08 10:46:08,424] WARNING: ALERT - finalTest.mp4 Frame:1 Violations:['NO-Safety Vest', 'NO-Mask', 'NO-Mask'] Timestamp:2025-11-08T10:46:08.424501+00:00
[2025-11-08 10:46:17,706] WARNING: ALERT - finalTest.mp4 Frame:29 Violations:['NO-Safety Vest', 'NO-Mask', 'NO-Mask', 'NO-Mask'] Timestamp:2025-11-08T10:46:17.706480+00:00
[2025-11-08 10:46:38,465] WARNING: ALERT - finalTest.mp4 Frame:91 Violations:['NO-Mask', 'NO-Mask', 'NO-Safety Vest'] Timestamp:2025-11-08T10:46:38.465153+00:00
[2025-11-08 10:46:38,812] WARNING: ALERT - finalTest.mp4 Frame:92 Violations:['NO-Mask', 'NO-Mask', 'NO-Safety Vest', 'NO-Safety Vest'] Timestamp:2025-11-08T10:46:38.812858+00:00
[2025-11-08 10:46:53,925] WARNING: ALERT - finalTest.mp4 Frame:139 Violations:['NO-Safety Vest', 'NO-Mask'] Timestamp:2025-11-08T10:46:53.925876+00:00
[2025-11-08 10:46:55,844] WARNING: ALERT - finalTest.mp4 Frame:145 Violations:['NO-Mask', 'NO-Mask', 'NO-Safety Vest', 'NO-Safety Vest', 'NO-Safety Vest'] Timestamp:2025-11-08T1


✅ Video processed and saved at: /content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/visual_demo/videos/alert_finalTest.mp4
📄 Alert logs stored in: /content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs


In [23]:
import os
print("EMAIL_USER:", os.getenv("EMAIL_USER"))
print("EMAIL_PASS:", "SET" if os.getenv("EMAIL_PASS") else "NOT SET")


EMAIL_USER: badabagni25r@gmail.com
EMAIL_PASS: SET


In [25]:
# ==========================================================
#  ALERT MANAGER CONFIGURATION + FUNCTIONS
# ==========================================================
import os, json, smtplib, logging
from datetime import datetime, timezone
from email.message import EmailMessage

# Set Gmail credentials as environment variables
os.environ["EMAIL_USER"] = "badabagni25r@gmail.com"   # <-- change this
os.environ["EMAIL_PASS"] = "yjcm lhxj kfap lsvq"       # <-- use Gmail App Password

# --- Configuration ---
config = {
    "enabled_channels": {
        "console": True,
        "email": True,
        "gui": False
    },
    "email": {
        "smtp_server": "smtp.gmail.com",
        "smtp_port": 587,
        "sender": os.getenv("EMAIL_USER"),
        "recipients": ["rohi131279@gmail.com"],
        "username": os.getenv("EMAIL_USER"),
        "password": os.getenv("EMAIL_PASS")
    },
    "suppression_window_seconds": 30,
    "outputs_dir": "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs",
    "log_csv": "alert_log.csv",
    "log_text": "alert_summary.txt",
    "alert_timeout_seconds": 2,
    "max_alerts_per_minute": 60
}

os.makedirs(config["outputs_dir"], exist_ok=True)

# ==========================================================
#  ALERT CHANNELS
# ==========================================================

def send_email_alert(violation_event, email_cfg):
    msg = EmailMessage()
    subject = f"PPE Violation: {', '.join(violation_event['violations'])} - {violation_event['video']}"
    body = f"""
⚠️ PPE Violation detected

Video: {violation_event['video']}
Frame: {violation_event['frame']}
Violations: {', '.join(violation_event['violations'])}
Timestamp (UTC): {violation_event['timestamp']}

-- Alert System
"""
    msg["From"] = email_cfg.get("sender")
    msg["To"] = ", ".join(email_cfg.get("recipients", []))
    msg["Subject"] = subject
    msg.set_content(body)

    try:
        with smtplib.SMTP(email_cfg["smtp_server"], email_cfg["smtp_port"], timeout=10) as server:
            server.starttls()
            server.login(email_cfg["username"], email_cfg["password"])
            server.send_message(msg)
        print(f"📧 Email sent for {violation_event['violations']} at frame {violation_event['frame']}")
        return {"status": "sent", "channel": "email", "ts": datetime.now(timezone.utc).isoformat()}
    except Exception as e:
        print(f"❌ Email failed: {e}")
        return {"status": "failed", "channel": "email", "error": str(e), "ts": datetime.now(timezone.utc).isoformat()}


# Console logging
logger = logging.getLogger("alert_system")
logger.setLevel(logging.INFO)
if not logger.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(asctime)s] %(levelname)s: %(message)s")
    ch.setFormatter(formatter)
    logger.addHandler(ch)

def send_console_alert(violation_event):
    ts = datetime.now(timezone.utc).isoformat()
    msg = f"🚨 ALERT - {violation_event['video']} Frame:{violation_event['frame']} Violations:{violation_event['violations']} Timestamp:{ts}"
    logger.warning(msg)
    return {"status": "sent", "channel": "console", "ts": ts}


# ==========================================================
#  ALERT MANAGER CLASS
# ==========================================================
import csv, time

class AlertManager:
    def __init__(self, cfg=config):
        self.cfg = cfg
        self.last_sent = {}
        self.csv_path = os.path.join(cfg["outputs_dir"], cfg["log_csv"])
        if not os.path.exists(self.csv_path):
            with open(self.csv_path, "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(["timestamp", "video", "frame", "violations", "channel", "status", "error"])

    def handle_violation(self, event):
        key = ",".join(sorted(event["violations"]))
        now = time.time()

        # Suppression window
        if key in self.last_sent and (now - self.last_sent[key]) < self.cfg["suppression_window_seconds"]:
            return

        self.last_sent[key] = now
        results = []

        # Console alert
        if self.cfg["enabled_channels"].get("console", False):
            results.append(send_console_alert(event))

        # Email alert
        if self.cfg["enabled_channels"].get("email", False):
            results.append(send_email_alert(event, self.cfg["email"]))

        # Log to CSV
        for r in results:
            with open(self.csv_path, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([
                    datetime.now(timezone.utc).isoformat(),
                    event["video"],
                    event["frame"],
                    ",".join(event["violations"]),
                    r.get("channel"),
                    r.get("status"),
                    r.get("error", "")
                ])


# ==========================================================
#  YOLO DETECTION + ALERT PIPELINE
# ==========================================================
from ultralytics import YOLO
import cv2

# Initialize model and alert system
model = YOLO("/content/drive/MyDrive/Project/Dataset/css-data/cross_validation/results_fold3/weights/best.pt")
alert_mgr = AlertManager()

# Input video
video_path = "/content/drive/MyDrive/Project/Dataset/finalTest.mp4"
base_name = os.path.basename(video_path)

# Output setup
output_dir = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/visual_demo/videos"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"alert_{base_name}")

cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

frame_id = 0
print(f"🎥 Processing video: {base_name}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame, verbose=False)

    annotated_frame = results[0].plot()
    names = model.names
    detections = [names[int(cls)] for cls in results[0].boxes.cls]

    # print detections once every 100 frames
    if frame_id % 100 == 0:
        print(f"Frame {frame_id}: Detections ->", detections)

    # Rule: Violations start with "NO-"
    violations = [v for v in detections if "NO-" in v]

    if violations:
        event = {
            "video": base_name,
            "frame": frame_id,
            "violations": violations,
            "timestamp": datetime.now(timezone.utc).isoformat()
        }
        alert_mgr.handle_violation(event)

    out.write(annotated_frame)

cap.release()
out.release()

print(f"\n✅ Video processed and saved at: {output_path}")
print(f"📄 Alert logs stored in: {alert_mgr.cfg['outputs_dir']}")


🎥 Processing video: finalTest.mp4


[2025-11-08 10:51:18,194] WARNING: 🚨 ALERT - finalTest.mp4 Frame:1 Violations:['NO-Safety Vest', 'NO-Mask', 'NO-Mask'] Timestamp:2025-11-08T10:51:18.194890+00:00


📧 Email sent for ['NO-Safety Vest', 'NO-Mask', 'NO-Mask'] at frame 1


[2025-11-08 10:51:28,158] WARNING: 🚨 ALERT - finalTest.mp4 Frame:29 Violations:['NO-Safety Vest', 'NO-Mask', 'NO-Mask', 'NO-Mask'] Timestamp:2025-11-08T10:51:28.158320+00:00


📧 Email sent for ['NO-Safety Vest', 'NO-Mask', 'NO-Mask', 'NO-Mask'] at frame 29


...

📧 Email sent for ['NO-Mask', 'NO-Mask', 'NO-Safety Vest', 'NO-Mask', 'NO-Safety Vest'] at frame 329


[2025-11-08 10:53:22,802] WARNING: 🚨 ALERT - finalTest.mp4 Frame:330 Violations:['NO-Mask', 'NO-Mask', 'NO-Safety Vest'] Timestamp:2025-11-08T10:53:22.802582+00:00


📧 Email sent for ['NO-Mask', 'NO-Mask', 'NO-Safety Vest'] at frame 330


[2025-11-08 10:53:25,749] WARNING: 🚨 ALERT - finalTest.mp4 Frame:337 Violations:['NO-Mask', 'NO-Mask', 'NO-Safety Vest', 'NO-Safety Vest'] Timestamp:2025-11-08T10:53:25.749560+00:00


📧 Email sent for ['NO-Mask', 'NO-Mask', 'NO-Safety Vest', 'NO-Safety Vest'] at frame 337

✅ Video processed and saved at: /content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/visual_demo/videos/alert_finalTest.mp4
📄 Alert logs stored in: /content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs
